In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from gensim.parsing.preprocessing import strip_non_alphanum, strip_punctuation
import torch
import torchtext
from sklearn.model_selection import *
from torch import nn
import gensim.downloader as api

from torchtext.data import Field, LabelField
from torchtext.data import BucketIterator
from torchtext.datasets import IMDB
from transformers import BertTokenizer
from transformers import BertModel
from skorch import NeuralNetClassifier
from skorch.callbacks import Freezer
from sklearn.ensemble import *
from sklearn.metrics import *
from sklearn.pipeline import * 
from sklearn.compose import *
from sklearn.linear_model import *
from sklearn.naive_bayes import *
from sklearn.neural_network import *
from sklearn.preprocessing import *
from skorch.callbacks import ProgressBar
from util import *
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import *
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import fse
from fse.models import uSIF

from tqdm.auto import tqdm
from tqdm.utils import CallbackIOWrapper
from skopt.space import Real, Integer
from skopt import BayesSearchCV

In [2]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [3]:
subset = list((set(tier1)-set(["PersonalizedProduct"]))&set(all_tiers_100))
subset

['AnalysisAndModeling',
 'SpecificationofUse',
 'Manufacturing',
 'SurgicalMethod',
 'Imaging',
 'AnatomicalTarget']

In [4]:
nice_subset = [tier_translations[x]  for x in subset]
nice_subset

['Analysis and Modeling',
 'Specification of Use',
 'Manufacturing',
 'Surgical Method',
 'Imaging',
 'Anatomical Target']

In [5]:
prefixed_subset = [f"This is labelled: {x}" for x in nice_subset]
prefixed_subset

['This is labelled: Analysis and Modeling',
 'This is labelled: Specification of Use',
 'This is labelled: Manufacturing',
 'This is labelled: Surgical Method',
 'This is labelled: Imaging',
 'This is labelled: Anatomical Target']

In [6]:
training_labels = training_set[subset]
testing_labels = testing_set[subset]

In [95]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("/var/patentmark/transformer-training/patent-electra-v4-patent-mnr-cl/")

In [96]:
embedded_subset = model.encode(prefixed_subset)

In [97]:
training_embeddings = model.encode(training_set.abstract.values)

In [98]:
from sklearn.metrics.pairwise import cosine_similarity

In [99]:
testing_embeddings = model.encode(testing_set.abstract.values)

In [100]:
labeling = cosine_similarity(testing_embeddings, embedded_subset)

In [101]:
testing_embeddings.shape

(243, 768)

In [102]:
embedded_subset.shape

(6, 768)

In [103]:
testing_labels.head()

,AnalysisAndModeling,SpecificationofUse,Manufacturing,SurgicalMethod,Imaging,AnatomicalTarget
0,True,False,True,False,False,True
1,True,False,False,False,True,True
2,False,False,False,True,False,True
3,False,True,False,False,True,False
4,True,False,True,False,True,True


In [104]:
labeling[0] 
# ['LABEL: Manufacturing',
# 'LABEL: Analysis and Modeling',
# 'LABEL: Imaging',
# 'LABEL: Anatomical Target',
# 'LABEL: Surgical Method',
 #'LABEL: Specification of Use']

array([0.6874917 , 0.7520376 , 0.75544107, 0.76167053, 0.74262685,
       0.8164581 ], dtype=float32)

In [105]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
classifier = RandomForestClassifier() #KNeighborsClassifier(n_neighbors=5, n_jobs=-1)

In [106]:
classifier.fit(training_embeddings, training_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [107]:
predictions = classifier.predict(testing_embeddings)

In [108]:
print(classification_report(testing_labels, predictions, target_names=subset))

                     precision    recall  f1-score   support

AnalysisAndModeling       0.28      0.17      0.21        84
 SpecificationofUse       0.40      0.28      0.33        79
      Manufacturing       0.35      0.28      0.31        83
     SurgicalMethod       0.80      0.20      0.32        40
            Imaging       0.60      0.68      0.64       133
   AnatomicalTarget       0.69      0.82      0.75       164

          micro avg       0.56      0.50      0.53       583
          macro avg       0.52      0.40      0.43       583
       weighted avg       0.53      0.50      0.50       583
        samples avg       0.56      0.52      0.50       583



/home/martin/anaconda3/envs/phenetics/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/martin/anaconda3/envs/phenetics/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
